In [5]:
consumer_key = 'twKlZH3suYkRJRYBnJFGLCCnL'
consumer_secret = 'RBgW7DfW0FxcRl3VXm76tBJ02LmPVdQL8A5RoeMQrAF5lW61bq'
access_token = '107991822-bjjdRQGUGSRLMtQVnOnSPY8yqLOVrf81uWCMvHKy'
access_token_secret = 'V2lRDvXjveAh26FLhghMOt6aY0xWS9fARfDOCVOCs6sPg'

In [7]:
from tweepy import API
from tweepy.auth import OAuthHandler
from tweepy import Cursor
import csv
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)


api = API(auth)

# Open/Create a file to append data
csvFile = open('result-deadpool-with-header.csv', 'a')
#Use csv Writer
csvWriter = csv.writer(csvFile, lineterminator='\n')

for tweet in Cursor(api.search, q="#deadpoolmovie", lang='en', count='100000').items() :
   if(tweet.retweet_count > 10 & tweet.retweeted == False) : 
        try:
           csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8'), tweet.favorite_count])
        except:
            continue

In [69]:
import urllib
import urllib.request
import ast
data = urllib.parse.urlencode({"text": "I'm a very good boy "}) 
binary_data = data.encode('utf-8')
u = urllib.request.urlopen("http://text-processing.com/api/sentiment/", binary_data)
the_page = u.read()
print(the_page)

b'{"probability": {"neg": 0.22531846855219551, "neutral": 0.084284385065714951, "pos": 0.77468153144780449}, "label": "pos"}'


In [8]:
import urllib
import urllib.request
import json
import pandas as pd
import re

dataf = pd.read_csv('result-deadpool-with-header.csv')
results = []
for index, row in dataf.iterrows():
    print(row['tweet'])
    data = urllib.parse.urlencode({"text": re.sub(r"http\S+", "", row['tweet'])}) 
    binary_data = data.encode('utf-8')
    u = urllib.request.urlopen("http://text-processing.com/api/sentiment/", binary_data)
    result = json.loads(u.read().decode('utf-8'))
    results.append(result['label'])
    print(result['label'])
dataf['label-ext'] = results
dataf.to_csv('result-deadpool-labeled.csv')

b'RT @ThisIsA_Podcast: was @VancityReynolds penis in #Deadpool REAL or CGI? We discuss\n#PodernFamily #DeadpoolMovie\nhttps://t.co/j32OkHlwY4\nh\xe2\x80\xa6'
neg
b"RT @DianaGreenhalgh: #DeadpoolMovie is finally out! If you missed it, here's my official @deadpoolmovie poster! #deadpoolcore #SkullPoopL h\xe2\x80\xa6"
neg
b"RT @ScreenPrism: #DeadpoolMovie out today! What are #Deadpool's superpowers? https://t.co/dVH3pilOY6 #superhero #marvel @RyanReynolds https\xe2\x80\xa6"
neutral
b'RT @weoverachieve: The #funko #deadpool #deadpoolmovie #marvel #collectortcorps came today. This is the stuff we #giveaway to fans \xf0\x9f\x98\x9b https:\xe2\x80\xa6'
pos
b'RT @MiracoleBurns: @actorChrisBurns @deadpoolmovie @gstqfashions @BGZ_Studios because #DeadpoolMovie &amp; guns! https://t.co/m2sY1uZYUO'
neutral
b'RT @lizardoffire: "Mmm I\'m so touching myself tonight" #DeadpoolMovie https://t.co/BjkvXOE3co'
pos
b'RT @ThisIsA_Podcast: was @VancityReynolds penis in #Deadpool REAL or CGI? We discuss\n#Pod

In [11]:
testdf = pd.read_csv('result-deadpool-test.csv')
testdf = testdf[['tweet','label']]
tweets = []
for index, row in testdf.iterrows():
    words = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",row['tweet']).split())
    #words = re.sub(r"http\S+", "", row['tweet'])
    sentiment = row['label']
    words_filtered = [e.lower() for e in words.split() if len(e) >= 4]
    tweets.append((words_filtered, sentiment))
    
tweets


[(['deadpoolmovie', 'moviereview', 'what', 'think'], 'nuetral'),
 (['version', 'deadpoolmovie', 'the1stbat'], 'nuetral'),
 (['fair',
   'colossus',
   'keep',
   'around',
   'negasonic',
   'teenage',
   'warhead',
   'deadpoolmovie'],
  'pos'),
 (['snow', 'movie', 'probably', 'gonna', 'make', 'money'], 'pos'),
 (['snow', 'movie', 'probably', 'gonna', 'make', 'money'], 'pos'),
 (['beanstalk',
   'eyes',
   'read',
   'deadpool',
   'himself',
   'deadpool',
   'deadpoolmovie',
   'deadpoolquotes',
   'xmen',
   'marvel',
   'movie'],
  'pos'),
 (['eyes',
   'read',
   'deadpool',
   'himself',
   'deadpool',
   'deadpoolmovie',
   'deadpoolquotes',
   'xmen',
   'marvel',
   'moviereview'],
  'pos'),
 (['deadpool',
   'superhero',
   'comic',
   'cufflinks',
   'nerd',
   'geek',
   'craftsesh',
   'craftbuzz',
   'etsymntt',
   'shoppershour',
   'deadpo'],
  'neg'),
 (['bodypainted',
   'cosplay',
   'model',
   'copycat',
   'from',
   'deadpool',
   'photo',
   'blue',
   'line',


In [17]:
import nltk
from nltk.text import TextCollection
from collections import OrderedDict
#http://www.laurentluce.com/posts/twitter-sentiment-analysis-using-python-and-nltk/
def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features
def get_words_in_tweets(tweets):
    all_words = []
    for (words, sentiment) in tweets:
      all_words.extend(words)
    return all_words
def get_tfidf_word_features(tweets):
    tweets_text = [word for (word,sentiment) in tweets]
    tweets_coll = TextCollection(tweets_text)
    wordlist = get_words_in_tweets(tweets)
    wordlist_freq = nltk.FreqDist(wordlist)
    word_features = {}
    for word in set(wordlist):
        if word:
            wordlist_freq[word] =  (tweets_coll.idf(word)*wordlist_freq[word]) if(tweets_coll.idf(word)*wordlist_freq[word]) else 0
    return word_features.keys()
    
#The list of word features need to be extracted from the tweets. It is a list with every distinct words ordered 
#by frequency of appearance. We use the following function to get the list plus the two helper functions.
#word_features = get_word_features(get_words_in_tweets(tweets))
word_features = get_tfidf_word_features(tweets)

#To create a classifier, we need to decide what features are relevant.
#To do that, we first need a feature extractor. The one we are going to use returns a dictionary
#indicating what words are contained in the input passed. Here, the input is the tweet. 
#We use the word features list defined above along with the input to create the dictionary.
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

training_set = nltk.classify.apply_features(extract_features, tweets)
classifier = nltk.NaiveBayesClassifier.train(training_set)


nuetral
nuetral
pos
pos
pos
pos
pos
neg
pos
pos
nuetral
nuetral
nuetral
pos
pos
pos
pos
pos
nuetral
nuetral
nuetral
pos
pos
pos
pos
pos
ponuetrals
pos
pos
pos
pos
pos
pos
pos
neg
neg
pos
pos
pos
pos
pos
pos
pos
nuetral
pos
nuetral
nuetral
pos
pos
pos
pos
pos
pos
nuetral
nuetral
nuetral
pos
pos
pos
pos
nuetral
pos
pos
pos
pos
nuetral
nuetral
nuetral
nuetral
nuetral
nuetral
nuetral
pos
nuetral
nuetral
nuetral
nuetral
nuetral
nuetral
pos
nuetral
pos
pos
pos
pos
nuetral
nuetral
pos
pos
nuetral
pos
pos
pos
pos
pos
pos
nuetral
nuetral
pos
pos
nuetral
nuetral
nuetral
nuetral
nuetral
nuetral
nuetral
nuetral
nuetral
nuetral
nuetral
nuetral
nuetral
nuetral
nuetral
nuetral
pos
pos
pos
pos
pos
pos
nuetral
nuetral
pos
pos
pos
pos
pos
pos
nuetral
nuetral
nuetral
nuetral
nuetral
pos
pos
pos
nuetral
nuetral
nuetral
pos
pos
pos
pos
pos
pos
pos
nuetral
nuetral
pos
pos
pos
pos
pos
nuetral
nuetral
nuetral
pos
nuetral
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
nuetral
pos
pos
ponuetrals
nu

In [14]:
dataf = pd.read_csv('result-deadpool-with-header.csv')
df = pd.read_csv('result-deadpool-labeled.csv')
results2 = []
for index, row in dataf.iterrows():
    #results2.append(classifier.classify(extract_features(re.sub(r"http\S+", "", row['tweet']))))
    result = classifier.classify(extract_features(re.sub(r"http\S+", "", row['tweet'])))
    print (row['tweet'])
    print(result)
    results2.append(result)
df['classifier'] = results2
df.to_csv('result-deadpool-labeled.csv')

b'RT @ThisIsA_Podcast: was @VancityReynolds penis in #Deadpool REAL or CGI? We discuss\n#PodernFamily #DeadpoolMovie\nhttps://t.co/j32OkHlwY4\nh\xe2\x80\xa6'
pos
b"RT @DianaGreenhalgh: #DeadpoolMovie is finally out! If you missed it, here's my official @deadpoolmovie poster! #deadpoolcore #SkullPoopL h\xe2\x80\xa6"
pos
b"RT @ScreenPrism: #DeadpoolMovie out today! What are #Deadpool's superpowers? https://t.co/dVH3pilOY6 #superhero #marvel @RyanReynolds https\xe2\x80\xa6"
pos
b'RT @weoverachieve: The #funko #deadpool #deadpoolmovie #marvel #collectortcorps came today. This is the stuff we #giveaway to fans \xf0\x9f\x98\x9b https:\xe2\x80\xa6'
pos
b'RT @MiracoleBurns: @actorChrisBurns @deadpoolmovie @gstqfashions @BGZ_Studios because #DeadpoolMovie &amp; guns! https://t.co/m2sY1uZYUO'
pos
b'RT @lizardoffire: "Mmm I\'m so touching myself tonight" #DeadpoolMovie https://t.co/BjkvXOE3co'
pos
b'RT @ThisIsA_Podcast: was @VancityReynolds penis in #Deadpool REAL or CGI? We discuss\n#PodernFamil

In [15]:
df = pd.read_csv('result-deadpool-labeled.csv')
for index,row in df.iterrows():
    if(row['label-ext'] != row['classifier']):
        print(re.sub(r"http\S+", "", row['tweet']), row['label-ext'],  row['classifier'])

b'RT @ThisIsA_Podcast: was @VancityReynolds penis in #Deadpool REAL or CGI? We discuss\n#PodernFamily #DeadpoolMovie\n neg pos
b"RT @DianaGreenhalgh: #DeadpoolMovie is finally out! If you missed it, here's my official @deadpoolmovie poster! #deadpoolcore #SkullPoopL h\xe2\x80\xa6" neg pos
b"RT @ScreenPrism: #DeadpoolMovie out today! What are #Deadpool's superpowers?  #superhero #marvel @RyanReynolds  neutral pos
b'RT @MiracoleBurns: @actorChrisBurns @deadpoolmovie @gstqfashions @BGZ_Studios because #DeadpoolMovie &amp; guns!  neutral pos
b'RT @ThisIsA_Podcast: was @VancityReynolds penis in #Deadpool REAL or CGI? We discuss\n#PodernFamily #DeadpoolMovie\n neg pos
b'RT @ThisIsA_Podcast: was @VancityReynolds penis in #Deadpool REAL or CGI? We discuss\n#PodernFamily #DeadpoolMovie\n neg pos
b'RT @ThisIsA_Podcast: was @VancityReynolds penis in #Deadpool REAL or CGI? We discuss\n#PodernFamily #DeadpoolMovie\n neg pos
b'RT @ThisIsA_Podcast: was @VancityReynolds penis in #Deadpool REAL or CGI?